In [1]:
import numpy as np
import pandas as pd

In [32]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder

In [3]:
df = pd.read_csv('covid_toy.csv')

In [4]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df.isnull().sum()

,0
age,0
gender,0
fever,10
cough,0
city,0
has_covid,0


In [6]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],
                                                test_size=0.2)

In [7]:
X_train

,age,gender,fever,cough,city
1,27,Male,100.0,Mild,Delhi
85,16,Female,103.0,Mild,Bangalore
47,18,Female,104.0,Mild,Bangalore
20,12,Male,98.0,Strong,Bangalore
81,65,Male,99.0,Mild,Delhi
...,...,...,...,...,...
5,84,Female,NaN,Mild,Bangalore
26,19,Female,100.0,Mild,Kolkata
12,25,Female,99.0,Strong,Kolkata
18,64,Female,98.0,Mild,Bangalore


**1. Without using Column Transformer**

In [8]:
# adding simple imputer to fever col
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])

X_train_fever.shape

(80, 1)

In [9]:
# Ordinalencoding -> cough
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

X_train_cough.shape

(80, 1)

In [11]:
# OneHotEncoding -> gender,city
ohe = OneHotEncoder(drop='first',sparse_output=False)
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])

# also the test data
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

X_train_gender_city.shape

(80, 4)

In [12]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values

# also the test data
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

X_train_age.shape

(80, 1)

In [14]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)
# also the test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)

X_train_transformed.shape

(80, 7)

In [15]:
X_train_transformed

array([[ 27., 100.,   1.,   1.,   0.,   0.,   0.],
       [ 16., 103.,   0.,   0.,   0.,   0.,   0.],
       [ 18., 104.,   0.,   0.,   0.,   0.,   0.],
       [ 12.,  98.,   1.,   0.,   0.,   0.,   1.],
       [ 65.,  99.,   1.,   1.,   0.,   0.,   0.],
       [ 42., 104.,   1.,   0.,   0.,   1.,   0.],
       [ 19., 101.,   0.,   0.,   0.,   1.,   0.],
       [ 81., 101.,   0.,   0.,   0.,   1.,   0.],
       [  5., 102.,   1.,   0.,   1.,   0.,   0.],
       [ 34., 104.,   0.,   1.,   0.,   0.,   1.],
       [ 75., 104.,   0.,   1.,   0.,   0.,   1.],
       [ 50., 103.,   0.,   0.,   1.,   0.,   0.],
       [ 23.,  98.,   1.,   0.,   0.,   1.,   1.],
       [ 51., 104.,   1.,   0.,   0.,   0.,   0.],
       [ 40.,  98.,   0.,   1.,   0.,   0.,   1.],
       [ 34.,  98.,   1.,   0.,   1.,   0.,   1.],
       [ 60.,  99.,   0.,   0.,   0.,   1.,   0.],
       [ 73., 103.,   0.,   1.,   0.,   0.,   0.],
       [ 47., 100.,   0.,   0.,   0.,   0.,   1.],
       [ 51., 104.,   1.,   0.,

**2. Using Column Transformer**

In [16]:
from sklearn.compose import ColumnTransformer

In [18]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [24]:
X_train_transformed = transformer.fit_transform(X_train)
X_train_transformed.shape

(80, 7)

In [23]:
X_test_transformed = transformer.transform(X_test)
X_test_transformed.shape

(20, 7)

In [25]:
X_train_transformed

array([[100.,   0.,   1.,   1.,   0.,   0.,  27.],
       [103.,   0.,   0.,   0.,   0.,   0.,  16.],
       [104.,   0.,   0.,   0.,   0.,   0.,  18.],
       [ 98.,   1.,   1.,   0.,   0.,   0.,  12.],
       [ 99.,   0.,   1.,   1.,   0.,   0.,  65.],
       [104.,   0.,   1.,   0.,   0.,   1.,  42.],
       [101.,   0.,   0.,   0.,   0.,   1.,  19.],
       [101.,   0.,   0.,   0.,   0.,   1.,  81.],
       [102.,   0.,   1.,   0.,   1.,   0.,   5.],
       [104.,   1.,   0.,   1.,   0.,   0.,  34.],
       [104.,   1.,   0.,   1.,   0.,   0.,  75.],
       [103.,   0.,   0.,   0.,   1.,   0.,  50.],
       [ 98.,   1.,   1.,   0.,   0.,   1.,  23.],
       [104.,   0.,   1.,   0.,   0.,   0.,  51.],
       [ 98.,   1.,   0.,   1.,   0.,   0.,  40.],
       [ 98.,   1.,   1.,   0.,   1.,   0.,  34.],
       [ 99.,   0.,   0.,   0.,   0.,   1.,  60.],
       [103.,   0.,   0.,   1.,   0.,   0.,  73.],
       [100.,   1.,   0.,   0.,   0.,   0.,  47.],
       [104.,   0.,   1.,   0.,

In [28]:
customer = pd.read_csv('customer.csv')

In [29]:
customer.head()

,age,gender,review,education,purchased
0,30,Female,Average,School,No
1,68,Female,Poor,UG,No
2,70,Female,Good,PG,No
3,72,Female,Good,PG,No
4,16,Female,Average,UG,No


In [31]:
customer['education'].value_counts()

,count
education,
PG,18
School,16
UG,16


In [35]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',OrdinalEncoder(categories=[['Poor', 'Average', 'Good'],['School', 'UG', 'PG']]),['review','education']),
    ('tnf2',OneHotEncoder(sparse_output=False,drop='first'),['gender'])
],remainder='passthrough')

In [36]:
customer_transformed = transformer.fit_transform(customer)

In [37]:
customer_transformed.shape

(50, 5)

In [38]:
customer_transformed

array([[1.0, 0.0, 0.0, 30, 'No'],
       [0.0, 1.0, 0.0, 68, 'No'],
       [2.0, 2.0, 0.0, 70, 'No'],
       [2.0, 2.0, 0.0, 72, 'No'],
       [1.0, 1.0, 0.0, 16, 'No'],
       [1.0, 0.0, 0.0, 31, 'Yes'],
       [2.0, 0.0, 1.0, 18, 'No'],
       [0.0, 0.0, 0.0, 60, 'Yes'],
       [1.0, 1.0, 0.0, 65, 'No'],
       [2.0, 1.0, 1.0, 74, 'Yes'],
       [2.0, 1.0, 0.0, 98, 'Yes'],
       [2.0, 1.0, 1.0, 74, 'Yes'],
       [0.0, 0.0, 1.0, 51, 'No'],
       [1.0, 0.0, 0.0, 57, 'No'],
       [0.0, 2.0, 1.0, 15, 'Yes'],
       [0.0, 1.0, 1.0, 75, 'No'],
       [0.0, 1.0, 1.0, 59, 'Yes'],
       [0.0, 1.0, 0.0, 22, 'Yes'],
       [2.0, 0.0, 1.0, 19, 'No'],
       [0.0, 2.0, 1.0, 97, 'Yes'],
       [1.0, 0.0, 0.0, 57, 'Yes'],
       [1.0, 2.0, 1.0, 32, 'No'],
       [0.0, 2.0, 0.0, 18, 'Yes'],
       [2.0, 0.0, 0.0, 96, 'No'],
       [1.0, 2.0, 0.0, 16, 'Yes'],
       [2.0, 0.0, 0.0, 57, 'No'],
       [0.0, 2.0, 0.0, 53, 'No'],
       [0.0, 2.0, 0.0, 69, 'No'],
       [0.0, 0.0, 1.0, 48, 'No'],
  